<a href="https://colab.research.google.com/github/tuomaseerola/audio/blob/master/audio_corpus_analysis_tutorial2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music and Science – Audio Corpus Analysis Tutorial

[Tuomas Eerola](https://www.durham.ac.uk/staff/tuomas-eerola/), Durham University, Music Department, 2025.

# 1 Set up the libraries

First we activate few useful libraries (`numpy`, `matplotlib`) and then obtain features.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
%matplotlib inline

# 2 Obtain a dataset

Let's look at a classic genre categorization study and dataset by Tzanetakis & Cook (2002). The full data contains 100 audio excerpts from 10 different genres (1000 clips in total), but we are going to start with a smaller set to keep this light to run. It should be noted that the selection of the excerpts for this dataset for not particularly rigorous and represented the collection of music that George Tzanetakis had at his disposal at the time. And this dataset has some quirks and imperfections, but I think it is still a fun and illustrative example to explore.

Note. This dataset used to be available within `mirdata` library but has been removed. I have taken the acoustic features from kaggle, see [https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification](https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification) to allow this demo to be run (without audio).

In [ ]:
import pandas as pd

url = 'https://github.com/tuomaseerola/audio/blob/cf8017124d4886faa309f4892664fa3279a08274/features_30_sec.csv?raw=true'
df = pd.read_csv(url,index_col=0)
print(df.shape)
print(df.columns.values.tolist())

# 3 Extract features

Let's extract some features and use them to predict genres. We take some rhythm features, some timbral features, MFCCs (19 in total). In the features, these features have been calculated across the 30-second excerpt and then the mean and variance has been to represent the feature.

The summary of the track could be more sophisticated (one could take the median, and the standard deviation, for instance, or to avoid extreme values, or to take multiple measures from within each example, and using _bagging_ or _voting_ where separate clips are used in assessing the most likely genre. This latter goes close to [ensemble machine-learning techniques](https://en.wikipedia.org/wiki/Ensemble_learning)).




---
### Learning Task 1: Concepts?
---

I mentioned two concepts above, MFCCs and chromas. What are these [MFCCs](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum) and [chroma](https://en.wikipedia.org/wiki/Chroma_feature) features? And more importantly, how useful you think they will be for genre classification?


### 3.1 Explore features
Let's look at some of features across genres. Are there differences in **dynamics** or **brightness**?

In [ ]:
print(df.shape)
print(df.head(3))

In [ ]:
import seaborn as sns
sns.set_theme(style="ticks", color_codes=True)
fig, axes = plt.subplots(2,1, figsize=(12, 9))
sns.boxplot(ax=axes[0], x="label", y="rms_mean", data=df)
sns.boxplot(ax=axes[1], x="label", y="spectral_centroid_mean", data=df)
axes[0].set_title('RMS Across Genres')
axes[1].set_title('Spectral Centroid Across Genres')

---
### Learning Task 2: What are the feature differences between genres?
---
Can you describe the differences evident in the feature distributions? You can also change the features in the above plot (just type another feature to the section `y="rmse"`

## 3.2 Feature space

How large is our feature space and do have features that are redundant, that is highly similar to each other? This can be easily explored by visualising the correlations between all features.

In [ ]:
df2 = df.drop(['label','length'], axis = 1)
print(df2.shape)
corr = df2.corr() # Compute the correlation matrix
mask = np.triu(np.ones_like(corr, dtype=bool))       # Generate a mask for the upper triangle
f, ax = plt.subplots(figsize=(11, 9))                # Define matplotlib figure
cmap = sns.diverging_palette(230, 20, as_cmap=True)  # Custom colormap
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1.00, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}) # Draw the heatmap with the mask and correct aspect ratio

---
### Learning Task 2: Can we eliminate some redundant features?
---

Classification and machine-learning algorithms typically deal well with numerous features, but here we have only 100 observations and 39 variables, which is not a healthy proportion (too many variables compared to observations). Usually it is a good idea to have 10:1 or 15:1 or even 20:1 of observations to predictors. Based on the correlation matrix, what would you eliminate?


For intance, all chroma features have high positive correlations and some of the timbral features seem to be related. Let's trim the selection as we have quite a little data when using the mini dataset.

In [ ]:
df_trimmed = df2.iloc[:,0:17]
print(df_trimmed.shape)
print(df_trimmed.columns.values.tolist())

# 4 Classify with the features

We use all the features and relatively simple hierarchical classification tree model called [random forest](https://www.sciencedirect.com/topics/computer-science/random-forest-classifier). It creates a bunch of decision nodes based on the data by using a subset of the features and bootstrapping this process many times over. It is a robust technique and does not really whether the distribution are normal or not.

Before running the model, there are three operations to introduce that are part of the good practice for model construction.

**Cross-validation of the model**

We [cross-validate](https://scikit-learn.org/stable/modules/cross_validation.html) the model, which means that we split the data into _training_ and _testing sets_. We first train the model on the _training set_, which here is a randomly select 70% of the data. Once we have trained the model, we test it against the unseen data (_test set_, 30% of the data) to assess how the model performs.  This could be done by alterning the selection of the training and testing set and we could do this 10 times and average the results (this is called _k-fold cross-validation_).

**Stratifying the sample**

When we randomly split the data into training and testing sets, we might want to _stratify_ the data according to genre, which makes sure that we have similar proportion of examples from each genre at both sets.

**Normalize variables**

We also want to normalize the variables. This is not so crucial for the random forest model that we are going to use, but usually it is good idea to eliminate the differences the feature ranges have to the model. To normalize the variables, we turn them into z-scores, where the mean is 0 and standard deviation is 1.



In [ ]:
import pandas as pd
import sklearn as sk
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

X = df.drop('label', axis = 1)
Xn = preprocessing.normalize(X)
y = df['label']

test_size = 0.30 # taking 70:30 training and test set
seed = 9  # Random numbmer seeding for reapeatability of the code
X_train, X_test, y_train, y_test = train_test_split(Xn, y, test_size=test_size, random_state=seed,stratify=y)

RF = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=0).fit(X_train, y_train)
#RF.predict(X_test)
#print(round(RF.score(X_test, y_test), 4))
y_pred_test = RF.predict(X_test)


In [ ]:
print('Correct classification rate:',round(RF.score(X_test, y_test), 4))

The correct classification seems to be 69%.  

---
### Learning Task 3: Is that a good classification accuracy?
---

In order to answer this question, you should think what a model that predicts nonsense would achieve by chance? You could also check how this model compares to the work published by Tzanetakis (the paper is uploaded to Blackboard Ultra). Finally, might want to consider what is level of accuracy expected from listeners and there might be even studies about this to give you a benchmark.

# 5 Analyse the model

Let's look at this model in more detail and try to see which features are doing the most heavy lifting here and could we simplify the model and what kind of mistakes does the classification model make.

## 5.1 Visualise confusion matrix

Let's explore what kind of mistakes the model makes. Confusion matrix is a useful way to visualise this.

In [ ]:
import seaborn as sns

# Reshape
matrix = confusion_matrix(y_test, y_pred_test)
matrix = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]

# Build the plot
plt.figure(figsize=(16,7))
sns.set(font_scale=1.4)
sns.heatmap(matrix, annot=True, annot_kws={'size':10},
            cmap=plt.cm.Blues, linewidths=0.2)

# Add labels to the plot
class_names = RF.classes_ #np.unique(y_test)
tick_marks = np.arange(len(class_names))
tick_marks2 = tick_marks + 0.5
plt.xticks(tick_marks+0.5, class_names, rotation=90)
plt.yticks(tick_marks2, class_names, rotation=0)
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix for Random Forest Model')
plt.show()


## 5.2 Feature importance

Let's plot the feature importance from random forest classification.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

importance = RF.feature_importances_
n = X.columns[0:len(X.columns)]
im = pd.DataFrame({'data': importance,'names': n})
im2 = im.sort_values(by='data',ascending=False)
# plot feature importance
fig, ax = plt.subplots(figsize=(18, 5))
#figure(figsize=(10, 5))
plt.scatter(im2.names[0:9],im2.data[0:9],color='red')
plt.plot(im2.names[0:9],im2.data[0:9])
ax.set_title('10 strongest features')
plt.show()

The plot show the best 10 features and the first four seem to bring greater benefit to the model.

## 5.3 Simplify model

What happens if we take the four best features and try building a simpler model with these features?

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

X2 = X.filter(['mfcc1_mean','perceptr_var', 'mfcc2_mean', 'mfcc4_mean'])

test_size = 0.30 # taking 70:30 training and test set
seed = 2022  # Random numbmer seeding for reapeatability of the code
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=test_size, random_state=seed,stratify=y)

RF = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=seed).fit(X_train, y_train)
RF.predict(X_test)
# Make predictions for the test set
y_pred_test = RF.predict(X_test)
print(round(RF.score(X_test, y_test), 4))

---
### Learning Task 4: How good is the simplified model?
---

What do you think about the simplified model with 5 features? Is the model still good? You could look at the confusion to see what kind of mistakes the slimmer model starts to make.

There is concept call _principle of parsimony_ or the idea behind that simpler models are more parsimonius than complex models, which stems from _Occam’s razor_. There are several statistical measures that assess the model fit and parsimoniousness ([Akaike Information Criterion](https://en.wikipedia.org/wiki/Akaike_information_criterion) etc.). We are not entering into those calculations here but usually it is better to have a simple model and compromise the model accuracy a little bit than to gain few points in accuracy but having a complex model.

## 5.4 Summary

There numerous other algorithms to classify the materials, SVMs ([Support Vector Machines](https://scikit-learn.org/stable/modules/svm.html)), [K-nearest neighbour models (KNNs)](https://scikit-learn.org/stable/modules/neighbors.html), [Neural networks](https://scikit-learn.org/stable/modules/neural_networks_supervised.html), and many others.

We could have focussed more on features, their calculation, the summary measures, and subsets, but overall we achieved a good success with a small set of features. We have to remember that this is a mini-version of the original dataset. You are welcome to try how the full dataset would improve the results.

This process is pretty generic for all kinds of classification tasks, so the same procedure could be applied to prediction _emotion categories_, _meter_, _instrumentation_ and other properties of music.

# 6 References

- Tzanetakis, G. & Cook, P. (2002). Musical genre classification of audio signals. _IEEE Transactions on Speech and Audio Processing, 10(5)_, 293-302 [doi:10.1109/TSA.2002.800560](https://ieeexplore.ieee.org/document/1021072).